In [ ]:
#%pip install gensim

In [ ]:
#%pip install spacy

In [ ]:
#%pip install pyLDAvis 

In [46]:
#Imports
import pandas as pd
import pickle
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt

import gensim
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim import matutils

import spacy

import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [444]:
#Import soptify + genius dataset with song lyrics
df_lyrics = pd.read_pickle("df_rap_7k.pickle")

In [445]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_release_year,track_release_decade,artist_name,artist_genres,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_wordlen
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,2020,20,Quin Nfn,"[austin hip hop, southern hip hop, vapor trap]",257151,61,"Detroit Flow Lyrics\n(Ayy, DJ Showoutt, fuck g...",detroit flow fuck hold really like sound beats...,"[detroit, flow, fuck, hold, really, like, soun...",244
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,2018,10,Sons of Light,[],141,7,Pet Sound LyricsTurbid:\nYes it's Turbid and I...,pet sound yes turbid like reclaim freely like ...,"[pet, sound, yes, turbid, like, reclaim, freel...",209
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,2021,20,03 Greedo,"[cali rap, melodic rap, trap, vapor trap]",317202,71,"Substance (We Woke Up) Lyrics\nYeah, 03\nYeah,...",substance woke yeah yeah yeah yeah beat boy ta...,"[substance, woke, yeah, yeah, yeah, yeah, beat...",269
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,2017,10,Big Sean,"[detroit hip hop, hip hop, pop, pop rap, rap, ...",9276030,84,"Sacrifices Lyrics\nBrrrt!\nI look up, so many ...",look many shit many young metro trust gon shoo...,"[look, many, shit, many, young, metro, trust, ...",421
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,1995,90,Das EFX,"[east coast hip hop, gangster rap, hardcore hi...",301411,60,Hardcore Rap Act LyricsIntro/Hook (x2)\n(*The ...,hardcore rap act hook hardcore rap act back ha...,"[hardcore, rap, act, hook, hardcore, rap, act,...",267


In [446]:
#Create lemmatizer, stemmer and define stopwords

word_tokenizer=RegexpTokenizer(r'\w+')

wordnet_lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [447]:
#Save lyrics
temp = df_lyrics["lyrics_clean"].copy()

In [448]:
### Function to lemmatize tokenized documents (song lyrics here)
def lemmatizer(df):
    
    lemmed = []
    
    for song in df.lyrics_clean_split:
        lemmed.append([wordnet_lemmatizer.lemmatize(word) for word in song])
    return lemmed

In [449]:
### Fucntion to get dictionary and bag of words
#Returns dictionary and bow corpus
def get_dict_and_bow(data, extreme_list = None):
    
    #Create dictionary of words: mapping between word and id
    id2word = corpora.Dictionary(data)
    if extreme_list is not None:
        id2word.filter_extremes(no_below = extreme_list[0], no_above = extreme_list[1]) #Filter tokens that appear in less than and more than
    texts = data
    
    #Transform dictionary into bag of words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return id2word, corpus

In [450]:
### Function to get coherence score for particular model, dict and bow
#Returns the coherence model
def get_coherence(model, text, id_dict, method='c_v'):
    coherence_model = CoherenceModel(model=model, texts=text, dictionary=id_dict, coherence=method)
    coherence_lda = coherence_model.get_coherence()
    print(coherence_lda)
    return coherence_model

In [451]:
### Function to join document-topic scores back to document dataset
#Args: dataframe to join back into, corpus/bow, model to extract topics from 
def get_topics_in_df(df, corpus, model):

    #set minimum probability to 0 to ensure all topic scores are extracted
    get_document_topics = [model.get_document_topics(item,minimum_probability =0) for item in corpus]

    temp_df = []
    for i in range(len(get_document_topics)):
        temp_list = []
        for j in range(len(get_document_topics[i])):
            temp_tuple = get_document_topics[i][j]
            temp_list.append(temp_tuple[1])
        temp_df.append(temp_list)
    temp_df = pd.DataFrame(temp_df)
    df = df.join(temp_df)
    
    return df

In [452]:
lyrics_lemmatized_all = lemmatizer(df_lyrics)

In [453]:
#Create new column in df_lyrics with lemmed words
df_lyrics['lyrics_clean_split_lemmed'] = lyrics_lemmatized_all

In [295]:
id2word_all, corpus_all = get_dict_and_bow(lyrics_lemmatized_all, [200, 0.95])

In [272]:
#Fit LDA model
ntopics = 8
lda_model_all = LdaModel(corpus = corpus_all,
                    id2word = id2word_all,
                    num_topics = ntopics, 
                    random_state = 123,
                    passes=4,
                    alpha=0.01,
                    eta=0.01,
                    per_word_topics=True)

In [273]:
#Show the LDA topics: topic 0
lda_model_all.show_topic(topicid=0, topn=20)


[('nigga', 0.048973463),
 ('got', 0.031892452),
 ('like', 0.03175187),
 ('shit', 0.03075559),
 ('bitch', 0.028975658),
 ('fuck', 0.02635007),
 ('get', 0.025956139),
 ('back', 0.015310178),
 ('know', 0.0152687),
 ('hit', 0.010618327),
 ('gon', 0.010378368),
 ('money', 0.009442544),
 ('put', 0.008538144),
 ('as', 0.008126787),
 ('cause', 0.008063398),
 ('make', 0.0075593074),
 ('real', 0.0073576933),
 ('bout', 0.00725545),
 ('see', 0.006579687),
 ('say', 0.0062552476)]

In [274]:
#Get coherence score
coherence_model_all = get_coherence(lda_model_all, text = lyrics_lemmatized_all, id_dict=id2word_all)

0.4244097745908735


In [275]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_all = pyLDAvis.gensim_models.prepare(lda_model_all, corpus_all, id2word_all)
vis_all

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.067580  0.082882       1        1  19.211342
4     -0.058930 -0.021645       2        1  17.803064
3     -0.069040 -0.061194       3        1  15.903166
2     -0.072764  0.164767       4        1  11.994580
7     -0.077025 -0.021235       5        1  11.214077
6      0.330531  0.045233       6        1  10.230290
5      0.084577 -0.114481       7        1   9.437727
1     -0.069770 -0.074328       8        1   4.205753, topic_info=      Term          Freq         Total Category  logprob  loglift
400   feat  34706.000000  34706.000000  Default  30.0000  30.0000
173   yeah  27847.000000  27847.000000  Default  29.0000  29.0000
68     one  10099.000000  10099.000000  Default  28.0000  28.0000
64   nigga  14985.000000  14985.000000  Default  27.0000  27.0000
215   time  10209.000000  10209.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
51    like    662.854935  34321.477332   Topic8  -4.3892  -0.7783
55    make    440.538667   8276.825961   Topic8  -4.7978   0.2355
47    know    463.024131  18957.079470   Topic8  -4.7480  -0.5434
302    day    352.857094   4892.873734   Topic8  -5.0197   0.5392
157  right    332.812775   6077.175346   Topic8  -5.0782   0.2640

[478 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
874       1  0.039211    ace
874       2  0.060777    ace
874       3  0.001961    ace
874       4  0.021566    ace
874       5  0.001961    ace
...     ...       ...    ...
114       4  0.057218  young
114       5  0.049261  young
114       6  0.577084  young
114       7  0.068729  young
114       8  0.015405  young

[2498 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 8, 7, 6, 2])

In [276]:
#Name the topics
topic_names = {
    0:"Violence/Gangsta",
    1:"Musicality",
    2:"Struggle/Mortality/Spirituality",
    3:"Flex",
    4:"Romance/Sexuality",
    5:"Feats/Artists",
    6:"Black?",
    7:"Party"
}

In [277]:
#Join topics back to dataframe
df_lyrics_topics = df_lyrics.copy()
df_lyrics_topics = get_topics_in_df(df = df_lyrics, corpus = corpus_all, model = lda_model_all)

In [278]:
df_lyrics_topics.rename(columns = topic_names, inplace = True)

In [431]:
df_lyrics_topics.head(50)

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,lyrics_wordlen,lyrics_clean_split_lemmed,Violence/Gangsta,Musicality,Struggle/Mortality/Spirituality,Flex,Romance/Sexuality,Feats/Artists,Black?,Party
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,244,"[detroit, flow, fuck, hold, really, like, soun...",0.861611,0.026577,0.016043,0.000052,0.095562,0.000052,0.000052,0.000052
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,209,"[pet, sound, yes, turbid, like, reclaim, freel...",0.000083,0.029386,0.039138,0.000083,0.534294,0.247273,0.000083,0.149659
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,269,"[substance, woke, yeah, yeah, yeah, yeah, beat...",0.452609,0.010736,0.000047,0.000047,0.089101,0.000047,0.014476,0.432938
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,421,"[look, many, shit, many, young, metro, trust, ...",0.145224,0.062657,0.331924,0.275086,0.000030,0.024432,0.000030,0.160618
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,267,"[hardcore, rap, act, hook, hardcore, rap, act,...",0.240039,0.000050,0.000050,0.000050,0.759658,0.000050,0.000050,0.000050
5,6vmxWmPNjNo6fHQSYy6WWr,7DMH031vtzX0U1vsamPtl6,0auu2itHTxEdAMRHvx7CyG,246066,False,https://api.spotify.com/v1/tracks/6vmxWmPNjNo6...,Shine,10,track,0.759,...,201,"[shine, doubt, doubt, doubt, sewer, area, feel...",0.000071,0.243883,0.000071,0.155072,0.600690,0.000071,0.000071,0.000071
6,0H3eU0jb8xEziAE3BDn2Ke,4U2OAARLIOVPgJIlRbH03r,2SoPWaROkX78bTYU6RcQqW,227693,True,https://api.spotify.com/v1/tracks/0H3eU0jb8xEz...,Ashes To Ashes,11,track,0.638,...,330,"[old, school, underground, rap, early, scene, ...",0.000053,0.000053,0.000053,0.000053,0.445740,0.523573,0.030419,0.000053
7,5ygNO0E9Nido0T6oJ8WqDU,2ZGkTvwtwDoembcBr8SS0d,2IDLDx25HU1nQMKde4n61a,163353,True,https://api.spotify.com/v1/tracks/5ygNO0E9Nido...,No Worries,47,track,0.563,...,129,"[barely, hit, way, stop, get, deserve, call, k...",0.528297,0.000097,0.000097,0.371381,0.000097,0.000097,0.000097,0.099837
8,32zhF8yXc1lej5EcudJXp7,2VVvm4zJlUQm9XmBCvGN6z,3wyVrVrFCkukjdVIdirGVY,197209,True,https://api.spotify.com/v1/tracks/32zhF8yXc1le...,Back (feat. Lil Yachty),48,track,0.775,...,237,"[yachty, yachty, joe, feat, ray, yachty, bank,...",0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.999648,0.000050
9,1Bp5LdIxR3XlprIigqp4en,0z0z4DcXhHiobX5ZKAw8Qn,2rhFzFmezpnW82MNqEKVry,106160,True,https://api.spotify.com/v1/tracks/1Bp5LdIxR3Xl...,Adults Swim,57,track,0.894,...,188,"[swim, yeah, yeah, yeah, ooh, hah, woke, yes, ...",0.107429,0.000075,0.637452,0.000075,0.254743,0.000075,0.000075,0.000075


In [437]:
df_lyrics_topics.iloc[6258    ]

track_id                                                      7MCZ8P0QnG1Mci4TvGKbGQ
album_id                                                      5SnWHdZfyNbnvpMFbab8N5
artist_id                                                     0Yr4BBpK2dkCp2UsrJ9LZN
duration_ms                                                                   139076
flg_explicit                                                                    True
url                                https://api.spotify.com/v1/tracks/7MCZ8P0QnG1M...
track_name                                                             Die Right Now
track_popularity                                                                  49
track_type                                                                     track
track_danceability                                                             0.453
track_energy                                                                   0.534
track_key                                                        

In [434]:
df_lyrics_topics.Flex.sort_values(ascending=False)

5930    0.999753
3696    0.999652
6258    0.999517
2239    0.999458
2730    0.999458
          ...   
5585    0.000011
3913    0.000011
3058    0.000007
1264    0.000007
3817    0.000007
Name: Flex, Length: 6783, dtype: float32

In [221]:
#Export songs to pickle 
df_lyrics_topics.to_pickle("df_lyrics_topics.pickle")

# Topics for Nas Songs Only

In [454]:
df_nas = df_lyrics[(df_lyrics.artist_name == "Nas") ]

In [455]:
#& (df_lyrics.track_release_year >= 2020)

In [456]:
df_nas.track_release_year.value_counts()

1999    26
2021    26
2004    19
2007    18
2002    15
2014    13
2008    11
2001    11
1996    10
2012     9
2020     8
2019     7
1994     7
2003     5
2018     4
Name: track_release_year, dtype: int64

In [465]:
len(df_nas.lyrics_clean_split_lemmed.drop_duplicates(keep=False))

145

In [466]:
df_nas.drop_duplicates(keep=False)

TypeError: unhashable type: 'list'

In [400]:
len(df_nas)

43

In [401]:
text_nas = df_nas.lyrics_clean_split_lemmed.tolist()

In [402]:
#
id2word_nas, corpus_nas = get_dict_and_bow(text_nas)

In [403]:
ntopics = 6
lda_model_nas = LdaModel(corpus = corpus_nas,
                    id2word = id2word_nas,
                    num_topics = ntopics, 
                    random_state = 123,
                    passes=20,
                    alpha = 0.001,
                    eta = 0.01,
                    per_word_topics=True)

In [404]:
#Get coherence score
coherence_model_nas = get_coherence(lda_model_nas, text = text_nas, id_dict=id2word_nas)

0.30857427567991397


In [405]:
# Visualize the topics
vis_nas = pyLDAvis.gensim_models.prepare(lda_model_nas, corpus_nas, id2word_nas)
vis_nas

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.324265 -0.060856       1        1  40.144984
0      0.076972 -0.005494       2        1  24.539758
5      0.074647 -0.011702       3        1  18.401801
4      0.089130 -0.179382       4        1   8.017099
3      0.085652 -0.021631       5        1   7.160143
2     -0.002136  0.279065       6        1   1.736215, topic_info=       Term         Freq        Total Category  logprob  loglift
704    feat  1050.000000  1050.000000  Default  30.0000  30.0000
200     one   113.000000   113.000000  Default  29.0000  29.0000
68     time   106.000000   106.000000  Default  28.0000  28.0000
724    mind    49.000000    49.000000  Default  27.0000  27.0000
49    nigga   113.000000   113.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
197   never     1.854366    56.401176   Topic6  -4.9354   0.6385
40      man     1.840966   100.012866   Topic6  -4.9426   0.0585
323    like     1.840822   185.362473   Topic6  -4.9427  -0.5586
1954    job     0.935477     2.940058   Topic6  -5.6196   2.9083
944    need     0.938480    20.804682   Topic6  -5.6164   0.9548

[431 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
680       1  1.003111       action
682       1  1.000290  affirmative
1580      2  0.770854       always
1580      3  0.231256       always
2049      3  0.979481    architect
...     ...       ...          ...
77        4  0.064027         yeah
77        5  0.064027         yeah
77        6  0.170737         yeah
564       3  0.988930          yes
565       5  0.994941        youth

[727 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 6, 5, 4, 3])